In [10]:
import pandas as pd 
import numpy as np

TWEET_DATA = pd.read_csv("output.csv", usecols=["tweet_clean", "Tweet"])
TWEET_DATA.columns = ["label", "tweet"]

TWEET_DATA.head()

,label,tweet
0,[],NaN
1,"['abang', 'nggak', 'kenal', 'bimantara', 'hump...",abang nggak kenal bimantara humpuss citra lamt...
2,"['abunawas', 'laen', 'jokowi boong', 'g', 'mer...",abunawas laen jokowi boong g merugika rakyat j...
3,"['abunawas', 'pas']",abunawas pas
4,"['adil', 'pertanggungjawaban', 'buka']",adil pertanggungjawaban buka


In [11]:
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
TWEET_DATA["tweet_join"] = TWEET_DATA["label"].apply(join_text_list)

TWEET_DATA["tweet_join"].head()

0                                                     
1    abang nggak kenal bimantara humpuss citra lamt...
2    abunawas laen jokowi boong g merugika rakyat j...
3                                         abunawas pas
4                         adil pertanggungjawaban buka
Name: tweet_join, dtype: object

In [12]:

from sklearn.feature_extraction.text import TfidfVectorizer

# banyaknya term yang akan digunakan, 
# di pilih berdasarkan top max_features 
# yang diurutkan berdasarkan term frequency seluruh corpus
max_features = 1000

# Feature Engineering 
print ("------- TF-IDF on Tweet data -------")

tf_idf = TfidfVectorizer(max_features=max_features, binary=True)
tfidf_mat = tf_idf.fit_transform(TWEET_DATA["tweet_join"]).toarray()

print("TF-IDF ", type(tfidf_mat), tfidf_mat.shape)



------- TF-IDF on Tweet data -------
TF-IDF  <class 'numpy.ndarray'> (900, 1000)


In [13]:

terms = tf_idf.get_feature_names()

# sum tfidf frequency of each term through documents
sums = tfidf_mat.sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append((term, sums[col] ))

ranking = pd.DataFrame(data, columns=['term','rank'])
ranking.sort_values('rank', ascending=False)

c:\Users\Shortfire13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,rank
368,jokowi,60.532019
995,yg,32.018900
97,bbm,22.054337
727,presiden,21.897250
756,rakyat,18.311676
...,...,...
418,kelompok,0.511324
506,leluasa,0.508125
34,ane,0.495854
441,klaim,0.495854


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 1000

# calc TF vector
cvect = CountVectorizer(max_features=max_features)
TF_vector = cvect.fit_transform(TWEET_DATA["tweet_join"])

# normalize TF vector
normalized_TF_vector = normalize(TF_vector, norm='l1', axis=1)

# calc IDF
tfidf = TfidfVectorizer(max_features=max_features, smooth_idf=False)
tfs = tfidf.fit_transform(TWEET_DATA["tweet_join"])
IDF_vector = tfidf.idf_

# hitung TF x IDF sehingga dihasilkan TFIDF matrix / vector
tfidf_mat = normalized_TF_vector.multiply(IDF_vector).toarray()
tfidf.get_feature_names()



c:\Users\Shortfire13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aamiin',
 'abah',
 'abai',
 'abang',
 'abdullah',
 'abis',
 'abu',
 'abunawas',
 'acu',
 'ad',
 'ada',
 'adab',
 'adil',
 'agama',
 'ah',
 'ahok',
 'aja',
 'ajak',
 'ajar',
 'akhlak',
 'akidah',
 'aksi',
 'aku',
 'akun',
 'alam',
 'alas',
 'alias',
 'allah',
 'ama',
 'aman',
 'amanah',
 'amat',
 'ambil',
 'ambisi',
 'amburadul',
 'amin',
 'ampas',
 'amplop',
 'anak',
 'anarkis',
 'anas',
 'ancam',
 'ancur',
 'ane',
 'anggap',
 'anggar',
 'anggota',
 'angkat',
 'angkut',
 'anies',
 'anis',
 'anjing',
 'anti',
 'antisipasi',
 'apa',
 'aparat',
 'api',
 'aplikasi',
 'aq',
 'arab',
 'asa',
 'aset',
 'asing',
 'asli',
 'asn',
 'assalamualaikum',
 'asu',
 'asusila',
 'atas',
 'atur',
 'awal',
 'ayam',
 'ayo',
 'azwar',
 'baca',
 'bagus',
 'bahan',
 'bahas',
 'bahasa',
 'baik',
 'banding',
 'bandingin',
 'bang',
 'banget',
 'bangkitbersamaet',
 'bangsa',
 'bangun',
 'bansos',
 'bantu',
 'banyuwangi',
 'baris',
 'batalkankenaikanbbm',
 'batas',
 'batu',
 'bawa',
 'bawah',
 'bayar',
 'bbm',
 

In [15]:

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.preprocessing import normalize

# max_features = 1000

# # ngram_range (1, 3) to use unigram, bigram, trigram
# cvect = CountVectorizer(max_features=max_features, ngram_range=(1,3))
# counts = cvect.fit_transform(TWEET_DATA["tweet_join"])

# normalized_counts = normalize(counts, norm='l1', axis=1)

# tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(1,3), smooth_idf=False)
# tfs = tfidf.fit_transform(TWEET_DATA["tweet_join"])

# tfidf_mat = normalized_counts.multiply(tfidf.idf_).toarray()
# tfidf.get_feature_names()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 1000


def generate_tfidf_mat(min_gram, max_gram):
    cvect = CountVectorizer(max_features=max_features, ngram_range=(min_gram, max_gram))
    counts = cvect.fit_transform(TWEET_DATA["tweet_join"])

    normalized_counts = normalize(counts, norm='l1', axis=1)

    tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(min_gram, max_gram), smooth_idf=False)
    tfs = tfidf.fit_transform(TWEET_DATA["tweet_join"])

    tfidf_mat = normalized_counts.multiply(tfidf.idf_).toarray()
    
    TF = normalized_counts.toarray()
    IDF = tfidf.idf_
    TF_IDF = tfidf_mat
    return TF, IDF, TF_IDF, tfidf.get_feature_names()

# ngram_range (1, 1) to use unigram only
tf_mat_unigram, idf_mat_unigram, tfidf_mat_unigram, terms_unigram = generate_tfidf_mat(1,1)

# ngram_range (2, 2) to use bigram only
tf_mat_bigram, idf_mat_bigram, tfidf_mat_bigram, terms_bigram = generate_tfidf_mat(2,2)

# ngram_range (3, 3) to use trigram only
tf_mat_trigram, idf_mat_trigram, tfidf_mat_trigram, terms_trigram = generate_tfidf_mat(3,3)

# ---------- check sparse data -------------------
idx_sample = 1

print("Show TFIDF sample ke-" + str(idx_sample), "\n")
print(TWEET_DATA["tweet"][idx_sample], "\n")

print("\t\t\t", "TF", "\t\t", "IDF", "\t\t", "TF-IDF", "\t", "Term\n")
for i, item in enumerate(zip(tf_mat_unigram[idx_sample], idf_mat_unigram, tfidf_mat_unigram[idx_sample], terms_unigram)):
    if(item[2] != 0.0):
        print ("array position " + str(i) + "\t", 
               "%.6f" % item[0], "\t", 
               "%.6f" % item[1], "\t", 
               "%.6f" % item[2], "\t", 
               item[3])

Show TFIDF sample ke-1 

abang nggak kenal bimantara humpuss citra lamtoro gung anak jokowi nggak ujung kuku anak soeharto bang jaga bang anak keluarga jokowi seret kkn sila kritik 

			 TF 		 IDF 		 TF-IDF 	 Term

array position 3	 0.062500 	 7.109248 	 0.444328 	 abang
array position 38	 0.187500 	 4.912023 	 0.921004 	 anak
array position 82	 0.125000 	 6.703782 	 0.837973 	 bang
array position 190	 0.062500 	 6.416100 	 0.401006 	 citra
array position 329	 0.062500 	 5.856485 	 0.366030 	 jaga
array position 360	 0.125000 	 1.836248 	 0.229531 	 jokowi
array position 412	 0.062500 	 5.856485 	 0.366030 	 keluarga
array position 417	 0.062500 	 6.703782 	 0.418986 	 kenal
array position 468	 0.062500 	 7.109248 	 0.444328 	 kritik
array position 636	 0.125000 	 5.856485 	 0.732061 	 nggak
array position 857	 0.062500 	 6.703782 	 0.418986 	 soeharto


In [17]:
def get_TF_unigram(row):
    idx = row.name
    return [tf for tf in tf_mat_unigram[idx] if tf != 0.0]

TWEET_DATA["TF_UNIGRAM"] = TWEET_DATA.apply(get_TF_unigram, axis=1)

def get_IDF_unigram(row):
    idx = row.name
    return [item[1] for item in zip(tf_mat_unigram[idx], idf_mat_unigram) if item[0] != 0.0]

TWEET_DATA["IDF_UNIGRAM"] = TWEET_DATA.apply(get_IDF_unigram, axis=1)

def get_TFIDF_unigram(row):
    idx = row.name
    return [tfidf for tfidf in tfidf_mat_unigram[idx] if tfidf != 0.0]

TWEET_DATA["TFIDF_UNIGRAM"] = TWEET_DATA.apply(get_TFIDF_unigram, axis=1)

TWEET_DATA[["label", "TF_UNIGRAM", "IDF_UNIGRAM", "TFIDF_UNIGRAM"]].head()

# save TFIDF Unigram to Excel

TWEET_DATA[["label", "TF_UNIGRAM", "IDF_UNIGRAM", "TFIDF_UNIGRAM"]].to_csv("TFIDF_Unigram.csv", index=False)
